# Hyperparameter Tuning

The first part of this notebook is a condensed version of what we did in `4-Improve-Performance.ipynb`

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import PIL
from PIL import Image
import random
from sklearn.model_selection import train_test_split
import time

import torch
from torch import nn
import torch.multiprocessing as mp
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms as T
from torchvision.io import read_image
import torchvision.models as models
from torchvision.transforms import Lambda

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

img_dir = 'data'
cache_dir = '/domino/datasets/' + os.environ['DOMINO_STARTING_USERNAME'] + '/' + os.environ['DOMINO_PROJECT_NAME'] + '/scratch'
annotations_file = 'reduced_images.csv'

labels = pd.read_csv(annotations_file)
species = sorted(labels['question__species'].unique())
species_to_idx = dict(zip(species,range(len(species))))
idx_to_species = {v: k for k, v in species_to_idx.items()}

class SnapshotSerengetiDataset(Dataset):
    def __init__(self, annotations_file, img_dir, cache_dir, class_dict, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.cache_dir = cache_dir
        self.transform = transform
        self.target_transform = target_transform
        self.class2index = class_dict

    def __len__(self):
        return len(self.img_labels)
    
    def __cacheitem__(self, idx):
        cache_path = os.path.join(self.cache_dir, self.img_labels.iloc[idx, 0])
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        image = T.Resize(256)(image)
        torch.save(image, cache_path)
        
    def __getitem__(self, idx):
        cache_path = os.path.join(self.cache_dir, self.img_labels.iloc[idx, 0])
        if not os.path.isfile(cache_path):
            self.__cacheitem__(idx)
        image = torch.load(cache_path)
        image = torch.mul(image, 1/255.) # scale to [0, 1]
        label = self.class2index[self.img_labels.iloc[idx, 1]]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

train_transform = T.Compose([#T.Resize(256),
                             T.RandomRotation(30),
                             T.RandomHorizontalFlip(),
                             T.CenterCrop(224),
                             T.ConvertImageDtype(torch.float32),
                             T.Normalize(mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225])])

val_transform = T.Compose([#T.Resize(256),
                           T.CenterCrop(224),
                           T.ConvertImageDtype(torch.float32),
                           T.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])

target_transform = Lambda(lambda y: torch.zeros(len(species_to_idx), dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))

random_seed= 42
val_size = .1

def stratified_split(annotations_file, test_size = 0.2):
    img_labels = pd.read_csv(annotations_file)
    indices = img_labels.index
    labels = img_labels[['question__species']]
    train_indices, test_indices, _, _ = train_test_split(indices, labels, stratify=labels, test_size=test_size, random_state=random_seed)
    return train_indices, test_indices

train_indices, val_indices = stratified_split(annotations_file=annotations_file, test_size=val_size)

dataset_size = len(train_indices) + len(val_indices)

train_dataset = SnapshotSerengetiDataset(annotations_file=annotations_file, img_dir=img_dir, cache_dir=cache_dir, class_dict=species_to_idx, transform=train_transform, target_transform=target_transform)
val_dataset = SnapshotSerengetiDataset(annotations_file=annotations_file, img_dir=img_dir, cache_dir=cache_dir, class_dict=species_to_idx, transform=val_transform, target_transform=target_transform)

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

## Model Training Loop
Things to try:
* Unfreeze layers at a different epoch, or keep them frozen the whole time.

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        if epoch == 2:
            for child in model.children(): # Unfreeze layers
                for param in child.parameters():
                    param.requires_grad = True

        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.argmax(1))
            #if phase == 'train':
                #scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    score_dict = {
        'accuracy':[best_acc.item(),], 'optimizer':[optimizer,], 'scheduler':[scheduler,],
        'epochs':[num_epochs,], 'training_time':[time_elapsed,]
       }

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, score_dict

## Importing a model
Things to try:
* Different pretrained (or not pretrained) [PyTorch models](https://pytorch.org/vision/stable/models.html).
* Freezing different numbers of layers at the start.

Keep in mind that different models have different numbers of layers and different final layer names; you may need to do a Google search or read the PyTorch docs for the model to find these values if you switch from `resnet50`.

In [ ]:
model = models.resnet50(pretrained=True).to(device)
model.fc = nn.Linear(model.fc.in_features, len(species_to_idx)).to(device) # Rescale output fully-connected layer size
num_frozen_layers = 9

# Freeze `num_frozen_layers`
layer = 0
for child in model.children():
    layer += 1
    if layer <= num_frozen_layers:
        for param in child.parameters():
            param.requires_grad = False
print('Number of unfrozen layers: ' + str(layer-num_frozen_layers))

In [ ]:
for child in model.children():
    print(child)

## Train!
Things to try:
* Batch size. Effects ttaining speed and interacts with learning rate.
* Number of workers. See how this affects the training speed.
* The optimizer. `SGD` and `Adam` are preconfigured, but feel free to try [another one](https://pytorch.org/docs/stable/optim.html).



In [ ]:
num_workers=6
batch_size = 128

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=num_workers, pin_memory=True)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': dataset_size*(1-val_size), 'val': dataset_size*val_size}

loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9, weight_decay=5e-3)
#optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=5e-3)

scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Decays LR by a factor of 0.1 every 7 epochs

model, score_dict = train_model(model, loss_fn, optimizer, scheduler, num_epochs=5)

## Submit Result to Leaderboard
When you are happy with your model, run the following cell to submit your result to the leaderboard!

In [ ]:
import os
import csv
import time

import boto3
from botocore import UNSIGNED
from botocore.client import Config

user = os.environ['DOMINO_STARTING_USERNAME']
score_dict['user'] = [user,]
score_dict['batch_size'] = [batch_size,]

filename = 'tuning-results/' + user + '.csv'
pd.DataFrame(score_dict, index=[0,]).to_csv(filename, index=False)
    
time.sleep(0.5)

client = boto3.client('s3', aws_access_key_id='', aws_secret_access_key='')
client._request_signer.sign = (lambda *args, **kwargs: None)

client.upload_file(user + ".csv", "workshop-leaderboard", "lm-workshop/" + user + ".csv")

print("Data upload succesfully")
pd.read_csv(filename).head()